In [ ]:
import numpy as np
import struct

# Load the npy file
data = np.load('/mnt/data/xyzrgb.npy')

header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z rgb
SIZE 4 4 4 4
TYPE F F F F
COUNT 1 1 1 1
WIDTH {data.shape[0]}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {data.shape[0]}
DATA ascii
"""

def rgb_to_float(rgb):
    """Convert RGB to packed float format used in PCD."""
    rgb = rgb.astype(np.int64)
    packed = np.left_shift(rgb[0], 16) | np.left_shift(rgb[1], 8) | rgb[2]
    return float(struct.unpack('f', struct.pack('I', packed))[0])

pcd_content = header
for point in data:
    xyz = point[:3]
    rgb_float = rgb_to_float(point[3:6])
    pcd_content += f"{xyz[0]} {xyz[1]} {xyz[2]} {rgb_float}\n"


pcd_file_path = "/mnt/data/converted_pointcloud.pcd"
with open(pcd_file_path, 'w') as file:
    file.write(pcd_content)

pcd_file_path

In [3]:
import numpy as np
import open3d as o3d

# 1. Load the data
pcd = o3d.io.read_point_cloud('test_data/11881/converted_pointcloud.pcd')
data_xyzrgb = np.asarray(pcd.points)
labels = np.load('test_data/11881/label.npy')


color_map = {
    0: [1, 0, 0],  
    1: [0, 1, 0],  
}


colors = np.array([color_map[label] if label in color_map else [1, 1, 1] for label in labels])
pcd.colors = o3d.utility.Vector3dVector(colors)

vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0]) 
vis.run()
vis.destroy_window()